# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [59]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.


client = OpenAI()
llm_model = "gpt-3.5-turbo"


In [60]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content


In [61]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [62]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [63]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [64]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [65]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew. Is there anything else you would like to know or discuss?'

In [66]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Andrew. Is there anything else you would like to know or discuss?


In [67]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Andrew. Is there anything else you would like to know or discuss?"}

In [68]:
memory = ConversationBufferMemory()

In [69]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [70]:
print(memory.buffer)

Human: Hi
AI: What's up


In [71]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [72]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [73]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [74]:
from langchain.memory import ConversationBufferWindowMemory

In [75]:
memory = ConversationBufferWindowMemory(k=1)               

In [76]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [77]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [78]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [79]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [80]:
conversation.predict(input="What is 1+1?")

'1+1 equals 2. Is there anything else you would like to know?'

In [81]:
conversation.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to know?"

## ConversationTokenBufferMemory

In [82]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [83]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [84]:
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [85]:
from langchain.memory import ConversationSummaryBufferMemory

In [86]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [87]:
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events."}

In [88]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [89]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events.
Human: What would be a good demo to show?
AI:

> Finished chain.


"For the morning meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal. This could include a live demonstration of the language processing capabilities, the efficiency of the blockchain integration, and any new functionalities that have been developed. Additionally, for the lunch meeting with the customer interested in AI, a demo highlighting the AI algorithms and machine learning models used in our products could be impressive. It's important to tailor the demos to the specific interests and needs of each audience to make a strong impact."

In [90]:
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events, suggesting tailored demos for the product team and customer meetings. For the product team, showcasing the latest features and updates on LangChain is recommended, while highlighting AI algorithms and machine learning models for the customer meeting."}

In [92]:
conversation.predict(input="what should be the approximate duration of demo?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events, suggesting tailored demos for the product team and customer meetings. For the product team, showcasing the latest features and updates on LangChain is recommended, while highlighting AI algorithms and machine learning models for the customer meeting.
Human: what should be the approximate duration of demo?
AI:

> Finished chain.


"The approximate duration of the demo for the product team should be around 30 minutes to an hour, depending on the complexity of the features and updates you want to showcase. For the customer meeting interested in AI, a demo duration of 45 minutes to an hour would be suitable to cover the AI algorithms and machine learning models effectively. It's important to keep the demos engaging and informative to make a lasting impression on the audience."

In [93]:
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events, suggesting tailored demos for the product team and customer meetings. For the product team, showcasing the latest features and updates on LangChain is recommended, while highlighting AI algorithms and machine learning models for the customer meeting. The human inquires about the approximate duration of the demo.\nAI: The approximate duration of the demo for the product team should be around 30 minutes to an hour, depending on the complexity of the features and updates you want to showcase. For the customer meeting interested in AI, a demo duration of 45 minutes to an hour would be suitable to cover the AI algorithms and machine learning models effectively. It's importan